In [23]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [24]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
from keras.preprocessing import image #*

In [25]:
input_path = "data/"

In [26]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    input_path + 'train',
    batch_size=32,
    class_mode='binary',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    input_path + 'validation',
    shuffle=False,
    class_mode='binary',
    target_size=(224,224))

Found 2319 images belonging to 3 classes.
Found 574 images belonging to 3 classes.


In [27]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

C:\Users\tanon\AppData\Local\Continuum\anaconda3\envs\fastai\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [6]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(3, activation='softmax')(x)
model = Model(conv_base.input, predictions)

In [7]:
optimizer = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [8]:
STEP_SIZE_TRAIN=train_generator.n
STEP_SIZE_VALID=validation_generator.n

In [9]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=2319,  # added in Kaggle
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=574
                             )

Epoch 1/100
2319/2319 [==============================] - 1101s 475ms/step - loss: 0.0686 - accuracy: 0.9751 - val_loss: 0.0017 - val_accuracy: 0.8881
Epoch 2/100
2319/2319 [==============================] - 1129s 487ms/step - loss: 0.0159 - accuracy: 0.9947 - val_loss: 0.1685 - val_accuracy: 0.9231
Epoch 3/100
2319/2319 [==============================] - 1157s 499ms/step - loss: 0.0118 - accuracy: 0.9961 - val_loss: 0.4898 - val_accuracy: 0.8621
Epoch 4/100
2319/2319 [==============================] - 1164s 502ms/step - loss: 0.0095 - accuracy: 0.9965 - val_loss: 1.3197 - val_accuracy: 0.8953
Epoch 5/100
2319/2319 [==============================] - 1117s 482ms/step - loss: 0.0055 - accuracy: 0.9976 - val_loss: 2.2830 - val_accuracy: 0.8867
Epoch 6/100
2318/2319 [============================>.] - ETA: 0s - loss: 0.0067 - accuracy: 0.9974

In [0]:
!mkdir models
!mkdir models/keras

In [0]:
# save
model.save('models/keras/model.h5')

In [18]:
# load
model = load_model('models/keras/model.h5')

In [10]:
# save
model.save_weights('models/keras/weights.h5')
with open('models/keras/architecture.json', 'w') as f:
        f.write(model.to_json())

In [19]:
# # load
# with open('models/keras/architecture.json') as f:
#     model = model_from_json(f.read())
model.load_weights('models/keras/weights.h5')

In [28]:
validation_generator.class_indices

{'glass': 0, 'metal': 1, 'plastic': 2}

In [20]:
img_path = 'data/validation/glass/glass582.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

In [21]:
preds

array([[1.0000000e+00, 1.1792413e-30, 1.3864158e-21]], dtype=float32)

In [22]:
pip list


Package                  Version            
------------------------ -------------------
absl-py                  0.9.0              
asn1crypto               1.3.0              
astor                    0.8.1              
astroid                  2.3.3              
attrs                    19.3.0             
backcall                 0.1.0              
beautifulsoup4           4.8.2              
bleach                   3.1.0              
blis                     0.2.4              
Bottleneck               1.3.1              
CacheControl             0.12.6             
cachetools               4.0.0              
certifi                  2019.11.28         
cffi                     1.13.2             
chardet                  3.0.4              
cloudpickle              1.2.2              
colorama                 0.4.3              
contextlib2              0.6.0.post1        
cryptography             2.8                
cycler                   0.10.0             
cymem     